In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

# Using Tensorflow Keras instead of the original Keras

from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import seaborn as sns
import warnings

import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,BatchNormalization
import keras

import numpy as np
import pandas as pd
import warnings 
import tensorflow as tf # import tensor flow 
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import random
from keras.utils.np_utils import to_categorical 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
subs =  pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [ ]:
train.columns

In [ ]:
train.shape

In [ ]:
x_train = train.drop('label',axis=1)
y_train = train['label']

In [ ]:
x_train.head()

In [ ]:
y_train

In [ ]:
x_train.shape,y_train.shape

In [ ]:
test.shape

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.998):
            print("\nReached 99%accuracy so cancelling training!")
            self.model.stop_training = True

In [ ]:
x_train = x_train.astype('float32')
test = test.astype('float32')
x_train = x_train/225.0
test = test/225.0

x_train = x_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [ ]:
print(x_train.shape),print(test.shape)

In [ ]:
#model = tf.keras.models.Sequential([
 ##  tf.keras.layers.MaxPooling2D(2,2),
   # tf.keras.layers.Flatten(),    
    #tf.keras.layers.Dense(10,activation='softmax')
#])

model = Sequential()

model.add(Conv2D(64, kernel_size = 3, activation = "relu", padding="same", input_shape = (28, 28, 1)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size = 3, activation = "relu"))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size = 5, strides=2, activation = "relu", padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 3, activation = "relu"))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size = 3, activation = "relu"))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size = 5, strides=2, activation = "relu", padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 4, activation = "relu"))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.3))

model.add(Dense(units = 10, activation = "softmax"))

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(10, activation = "softmax"))
model.summary()


In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
from keras.optimizers import Adam, RMSprop
model.compile(optimizer = optimizer, loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) 

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='classifier.png', show_shapes=True, show_layer_names=True)
from IPython.display import Image
Image("classifier.png")

In [ ]:
epochs = 32
batch_size = 16

In [ ]:
model.fit(x_train,y_train,batch_size = batch_size,epochs=epochs, verbose = 2, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

In [ ]:
result = model.predict(test)

In [ ]:
y_pred = np.argmax(result, axis=1)
y_pred

In [ ]:
len(test)

In [ ]:
y_pred_df = pd.DataFrame({'ImageID': range(1, len(test)+1), 'Label': y_pred})

In [ ]:
y_pred_df

In [ ]:
y_pred_df.to_csv('/kaggle/working/Predictions.csv', index=False)